# **4.1 Compute the Parity of a Word**
---
---
- Parity of a binary word is `1` if the number of `1`'s in the word is odd
    - otherwise, it is `0`
    - Parity of 1011 = 1 
    - Parity of 1001 = 0 
- Parity Checks: used to detect single-bit errors in data storge and communication 
- ***Use a lookup table, not 2<sup>64</sup> entries***
---

### Brute-Force 
- iteratively test the value of each bit while tracking the number of `1`s seen so far
- Only care if number of `1`s is even or odd -> store number `mod 2`

In [1]:
def bf_parity(x: int) -> int:
    result = 0 
    while x:
        result ^= x&1 # result = result ^ (x & 1) 
        x >>= 1 # x = x >> 1 Dividing by power of 2 -> make sure its even 'mod 2'
    return result 

In [2]:
x = 14
y = 10

print(f"{x} as binary = {bin(x)} which parity = {bf_parity(x)}")
print(f"{y} as binary = {bin(y)} which parity = {bf_parity(y)}")

14 as binary = 0b1110 which parity = 1
10 as binary = 0b1010 which parity = 0


#### Time Complexity: `O(n)`
- n = word size 
---

## Erase Lowest Set Bit
##### roughly 20% faster than brute force 
- improve performance in the best and average cases 
- **`x&(x-1)`** = **x with lowest set bit erased**
    - x = (00101100)<sub>2</sub>
        - then: x-1 = (00101011)<sub>2</sub>
        - so: x&(x-1) = (00101100)<sub>2</sub> & (00101011)<sub>2</sub>
        - equals = (00101000)<sub>2</sub>
- **`x&~(x-1)`** **isolate the lowest bit that is `1` in `x`**

In [3]:
def erase_lowest_parity(x: int) -> int:
    result = 0 
    while x:
        result ^= 1 # result = result ^ 1 -> keeps track of evensVSodds
        x &= x - 1 # x = x&(x-1) -> erase lowest bit
    return result 

In [4]:
x = 14
y = 10

print(f"{x} as binary = {bin(x)} which parity = {erase_lowest_parity(x)}")
print(f"{y} as binary = {bin(y)} which parity = {erase_lowest_parity(y)}")

14 as binary = 0b1110 which parity = 1
10 as binary = 0b1010 which parity = 0


#### Time Complexity `O(k)`
- k = number of bits set to 1 in a word 
---

## Caching: Table Based
#### roughly four times faster than brute-force 
- cannot cache the parity of every 64-bit integer 
    - require 2<sup>64</sup> bits of storage (two exabytes)
- Computing Parity of a collection of bits -> does not matter how we group them 
    - aka the computation is **associative **
    - compute parity of a 64-bit integer:
        - grouping bits into four nonoverlapping 16-bit subwords 
        - computing parity of the subwords 
        - computing parity of the four subresults 
        - Why 16? -> 2<sup>16</sup> = 65536
            - relatively small and easier to cache the parity of all 16-bit words using an array
            - 16 also evenly divides 64

In [5]:
# Lookup Table for 2-bit words
# a, b, c, d = [00],[01],[10],[11]
cache1 = [0,1,1,0]
big_word = 11101010
# e, f, g, h = [11],[00],[10],[10]
cache2 = [0,0,1,1]

In [6]:
def binary2Int(binary):
    int_val, i, n = 0, 0, 0 
    
    while binary != 0:
        a = binary % 10
        int_val = int_val + a * pow(2,i)
        binary = binary//10
        i += 1
    return int_val

In [7]:
# Parity of First Two Bits
big_word = 11101010
wordy = binary2Int(big_word)

first_two = wordy >> 6
ftb = bin(first_two)
ft = int(ftb.replace('0b',''))
ft

11

In [8]:
# Parity of Next Two Bits -> does not remove leading (11) -> indexes other
big_word = 11101010
wordy = binary2Int(big_word)

second_two = wordy >> 4
s2b = bin(second_two)
s2 = int(s2b.replace('0b',''))
s2

1110

##### **MASK** -> used to extract the last two lines 
##### cannot index the cace with this -> leads to out-of-bounds access
- to get the last two bits after Right Shift by 4
    - bitwise-AND (00001110) with (00000011)

In [9]:
index_ft = ft & s2
idx_ftb = bin(index_ft)
idx_ft = int(idx_ftb.replace('0b',''))
idx_ft

10

### Time Complexity: `O(n/L)`
- `n` = word size 
- `L` = width of words we cache results with 
- `n/L` terms
- `O(1)` = time to do shifting 
- **does not include time for initialization of the lookup table**

--- 
## Exploit XOR Properties 
#### roghly 6 times faster than brute force 
- XOR is **associative** (does not matter how we grou bits)
- XOR is **commutative** (order in which we perform XORs does not change the result )
- parity of [b<sub>63</sub>, b<sub>62</sub>,...,b<sub>2</sub>, b<sub>1</sub>, b<sub>0</sub>]  **=**  parity of [b<sub>63</sub>,...,b<sub>32</sub>] and [b<sub>31</sub>,..., b<sub>0</sub>]
    - XOR of the two 32-bit values can be computed iwth a single shift and single 32-bit XOR instruction 
- leading bits are not meaningful -> want the least significant 

In [10]:
def parity(x: int) -> int:
    i = 64//2
    while i >= 1: 
        x ^= x >> i
        i = i//2
    return x & 0x1

In [11]:
big_word1 = 11101010
wordy1 = binary2Int(big_word1)
big_word0 = 111010101
wordy0 = binary2Int(big_word0)

print(parity(wordy1))
print(parity(wordy0))

1
0


In [12]:
def parity_book(x: int) -> int:
    x ^= x >> 32
    x ^= x >> 16
    x ^= x >> 8
    x ^= x >> 4
    x ^= x >> 2
    x ^= x >> 1
    return x & 0x1

In [13]:
big_word1 = 11101010
wordy1 = binary2Int(big_word1)
big_word0 = 111010101
wordy0 = binary2Int(big_word0)

print(parity_book(wordy1))
print(parity_book(wordy0))

1
0


### Time Complexity: `O(log n)`
- n = word size 

---
## Variant 
- do another day 